# Описание задачи
Данный Python notebook разбит на две логических части.

**Первая** – ознакомительная. Она демонстрирует, как осуществляется лемматизация – приведение слов в тексте к стандартной форме. Некоторые примеры представлены здесь. Для демонстрации используется библиотека Yandex MyStem. Ввиду того, что база данных, с которой мы работаем, очень большая, для проведения лемматизации недостаточно просто применить функцию mystem.lemmatize() к датафрейму: ее работа займет много часов, при этом наиболее вероятно, что в процессе закончится свободная оперативная память, и код завершится с ошибкой.
В комментариях в коде схематично описан алгоритм обработки датафрейма, выполнение которого не затратит слишком много времени и ресурсов (запускать данный код не требуется, но понять принцип работы желательно).

**Вторая часть кода** непосредственно связана с вашей задачей. Она загружает базу всех товаров (df_final) и выборку, сформированную по итогам вашей коллективной работы на первом этапе проекта (df_sample). Далее демонстрируется, как с помощью этой выборки можно обучить модели классификации (обычная логистическая регрессия и градиентный спуск с регуляризацией elastic net) и использовать полученную модель для out-of-sample классификации.


**Последовательность действий на данном этапе:**
1.	Ознакомиться с базовыми версиями моделей, предлагаемыми в данном ноутбуке. Они, ожидаемо, хорошо работают in-sample, но out-of-sample качество гораздо хуже
2.	Выбрать себе для анализа несколько товарных групп (3-5). Пожалуйста, используйте чат для координации, чтобы результаты получились более разнообразными. Справочник со всеми возможными товарами прилагаю в файле КИПЦ_веса.xlsx. Нас интересуют товары на семизначном уровне классификатора КИПЦ (например, 06.1.1.0.1.2, 01.1.4.4.4.1 и т. д.). По большинству товаров мы с вами набрали достаточно большую выборку, но лучше все равно перепроверить: задать targetProductCode в соответствующей ячейке и исполнить следующую за ней ячейку, посмотрев на сумму Y.sum(). У нас вряд ли получатся хорошие результаты, если товаров в группе меньше 30-40.
3.	Улучшить предлагаемые модели/предложить собственные. Вы можете, например, использовать иной классификатор (совсем не исчерпывающий список примеров есть, например, тут), улучшить работу моих классификаторов путем оптимизации гиперпараметров (l1_ratio, alpha и т.д.) через кросс-валидацию (примеры тут, тут, тут и в сотнях вопросов на stackoverflow)
4.	Сформировать единый датафрейм с результатами вашей классификации.


## 1. Lemmatizing data

In [ ]:
# install Python wrapper for Yandex MyStem
!pip install pymystem3
!pip install nltk
!pip install distributed
!pip install pyspark # takes 5-10 minutes
!pip install fastparquet
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# load other dependencies
import pandas as pd
import nltk
from nltk.corpus import stopwords
from string import punctuation
from joblib import Parallel, delayed
from tqdm import tqdm
import os
from distributed import Client
import dask.dataframe as dd
import pyspark
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# this should automatically download and install MyStem
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
# commonly used words which can be considered as "noise" and have to be removed
russian_stopwords = stopwords.words("russian")

In [ ]:
# custom function to perform lemmatization. Filters out stopwords, blank words,
# short words (<3 characters), punctuation and digits
def preprocess_text(text):
    tokens = m.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
               and token != " " \
               and len(token)>=3 \
               and token.strip() not in punctuation \
               and token.isdigit()==False]
    text = " ".join(tokens)
    return tokens

In [ ]:
# all the goods available in the database
df = pd.read_csv('all_goods.csv')
df.head(30)

,web_price_id,product_name,contributor_id,moment
0,6611458,ИГРОЛЕНД Игрушки интерактивные,1,2021-04-23T14:49:06.857Z
1,5318090,Calvin Klein Performance Футболка,1,2021-04-08T02:32:39.977Z
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,4,2021-11-15T20:21:20Z
3,6649792,Avocadoffka Кукла,4,2021-04-25T13:21:35Z
4,7420308,Uniqcute Постельное белье 2 спальный поплин,4,2021-06-13T06:52:33Z
5,5559609,Swiss Navy Лубрикант,4,2021-04-13T02:36:21Z
6,5487362,Lacoste Куртка,4,2021-04-13T01:37:18Z
7,7931249,Briari Платье,4,2021-07-08T07:30:11Z
8,2776174,4M Набор для опытов обучающий набор эксперимен...,1,2021-03-22T14:57:00.060Z
9,7420346,Flexpocket Держатель для масок,4,2021-06-13T06:47:38Z


In [ ]:
# download the necessary files from MyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# df['name_lemm'] = df['product_name'].apply(preprocess_text)
# the line above would be the standard way to lemmatize the data. However, due to internal restrictons of pymystem3 it takes
# several hours to run this single line of code.
#
# The reason why it takes so long is that for EACH row python makes a separete call to mystem.exe,
# which takes approx. 1 second to launch. The following code circumvents this restriction.
# 1. We use parallel computation (Client(n_workers=4)) so that the CPU of the computer is used more efficiently
# 2. We load the dataset all_goods.csv and break it int little pieces with df.repartition.
# They are saved in .parquet format in a separete folder
# 3. We apply the functon preprocess_df to each of these little pieces.
#
# preprocess_df() works in the following way:
# 1. We take the column with product names and transofrm it to a list prodnames.
# 2. Next we transform the list to a single string merged_text, where individual product names are separated with a key word "breakword"
# 3. Then we use preprocess_text() defined earlier to lemmatize and clear the product names.
# Using this function on a single string (rather than on a column of a dataframe) is several dozen times faster.
# 4. In a for loop we restore a list from the string and assign it to product_name_lemm

In [ ]:
def preprocess_df(df):
    m = Mystem()
    prodnames = df['product_name'].tolist()
    merged_text = " breakword ".join([str(x) for x in prodnames])
    doc = []
    res = []
    l = preprocess_text(merged_text)
    for t in l:
        if t != 'breakword':
            doc.append(t)
        else:
            res.append(doc)
            doc = []
    res.append(doc)
    res = [' '.join(x) for x in res]
    df['product_name_lemm'] = res
    m.close()
    return df
if __name__ == '__main__':
    client = Client(n_workers=4)

    # Splitting the big file
    df = dd.read_csv('all_goods.csv', dtype={'web_price_id': int, 'product_name': str, 'contributor_id': int, 'moment': str}) # Read a big data file
    df = df.repartition(npartitions=df.npartitions*100) # increase the value if the df is very large
    df.to_parquet('int/') # convert .csv file to .parquet parts

    # Loading the splitted file parts
    df = dd.read_parquet('int/')

    # Applying the function
    df = df.map_partitions(preprocess_df, meta={'web_price_id': int, 'product_name': str, 'contributor_id': int, 'moment': str, 'product_name_lemm' : str}) # Be sure not to '.compute' here

    df.to_parquet('preprocesed.parquet')
    client.close()

In [ ]:
df_final_dask = dd.read_parquet('preprocesed.parquet')
df_final_dask

In [ ]:
df_final = df_final_dask.compute()
df_final.head()

In [ ]:
df_final.to_csv('all_goods_lemm.csv', index=False)

## 2. Working with lemmatized data

In [ ]:
df_final = pd.read_csv('all_goods_lemm.csv')
df_final = df_final.drop(columns=['contributor_id', 'moment'])
df_final = df_final.drop_duplicates(subset=['web_price_id'])
df_final = df_final[~df_final.product_name_lemm.isnull()]
df_final = df_final[~df_final.product_name.isnull()]
df_final.head()

,web_price_id,product_name,product_name_lemm
0,6611458,ИГРОЛЕНД Игрушки интерактивные,игроленд игрушка интерактивный
1,5318090,Calvin Klein Performance Футболка,calvin klein performance футболка
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,шиповка nike цвет черный артикул rtlaao360101 ...
3,6649792,Avocadoffka Кукла,avocadoffka кукла
4,7420308,Uniqcute Постельное белье 2 спальный поплин,uniqcute постельный белье спальный поплин


In [ ]:
df_final

,web_price_id,product_name,product_name_lemm
0,6611458,ИГРОЛЕНД Игрушки интерактивные,игроленд игрушка интерактивный
1,5318090,Calvin Klein Performance Футболка,calvin klein performance футболка
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,шиповка nike цвет черный артикул rtlaao360101 ...
3,6649792,Avocadoffka Кукла,avocadoffka кукла
4,7420308,Uniqcute Постельное белье 2 спальный поплин,uniqcute постельный белье спальный поплин
...,...,...,...
9300025,7184397,Мойка кухонная Omoikiri Taki 54 U IF IN 4973046,мойка кухонный omoikiri taki
9300026,8682610,Перчатки Regatta цвет черный Артикул RE036DMJI...,перчатка regatta цвет черный артикул re036dmji...
9300028,10806102,R O C S Зубная щетка мягкая 1 шт,зубной щетка мягкий
9300029,9063186,Ацетилсалициловая кислота таблетки 500мг 30шт,ацетилсалициловый кислота таблетка 500мг 30шт


In [ ]:
df_sample = pd.read_csv('sample.csv', sep=';', dtype={'web_price_id': object, 'product_name': str, 'contributor_id': object, 'moment': str, 'rosstat_name': str})
df_sample = df_sample.drop(columns=['contributor_id', 'moment'])
df_sample = df_sample[~df_sample['web_price_id'].isnull()]
df_sample['web_price_id'] = df_sample['web_price_id'].astype(int)
df_sample = df_sample.drop_duplicates(subset=['web_price_id'])
df_sample = df_sample[df_sample['web_price_id'].isin(df_final['web_price_id'])]
df_sample = pd.merge(df_sample, df_final[['product_name_lemm', 'web_price_id']], on="web_price_id", how='left')
df_sample = df_sample[~df_sample.product_name_lemm.isnull()]
df_sample = df_sample[~df_sample.product_name.isnull()]
df_sample

,web_price_id,product_name,rosstat_name,product_name_lemm
0,8164323,КИСЕЯ Ткань для рукоделия бязь,"Ткани хлопчатобумажные бельевые, м",кисея ткань рукоделие бязь
1,8503735,1 Метр ткани Ткань для рукоделия бязь,"Ткани хлопчатобумажные бельевые, м",метр ткань ткань рукоделие бязь
2,7021870,Мегахлопок Ткань Бязь Валенсия 100х150 см,"Ткани хлопчатобумажные бельевые, м",мегахлопка ткань бязь валенсия 100х150
3,5293185,1 Метр ткани Ткань для рукоделия бязь,"Ткани хлопчатобумажные бельевые, м",метр ткань ткань рукоделие бязь
4,5293180,1 Метр ткани Ткань для рукоделия бязь,"Ткани хлопчатобумажные бельевые, м",метр ткань ткань рукоделие бязь
...,...,...,...,...
42190,8363100,ОрехМолл Бананы,"Бананы, кг",орехмолл банан
42191,7267302,ВКУСНО Бананы,"Бананы, кг",вкусно банан
42192,7622000,Фреш Продукт Бананы,"Бананы, кг",фреш продукт банан
42193,7399774,Казанские орехи Бананы,"Бананы, кг",казанский орех банан


In [ ]:
df = pd.merge(df_final, df_sample[['web_price_id', 'rosstat_name']], on='web_price_id', how='left')
df

,web_price_id,product_name,product_name_lemm,rosstat_name
0,6611458,ИГРОЛЕНД Игрушки интерактивные,игроленд игрушка интерактивный,NaN
1,5318090,Calvin Klein Performance Футболка,calvin klein performance футболка,NaN
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,шиповка nike цвет черный артикул rtlaao360101 ...,NaN
3,6649792,Avocadoffka Кукла,avocadoffka кукла,NaN
4,7420308,Uniqcute Постельное белье 2 спальный поплин,uniqcute постельный белье спальный поплин,NaN
...,...,...,...,...
4131967,7184397,Мойка кухонная Omoikiri Taki 54 U IF IN 4973046,мойка кухонный omoikiri taki,NaN
4131968,8682610,Перчатки Regatta цвет черный Артикул RE036DMJI...,перчатка regatta цвет черный артикул re036dmji...,NaN
4131969,10806102,R O C S Зубная щетка мягкая 1 шт,зубной щетка мягкий,NaN
4131970,9063186,Ацетилсалициловая кислота таблетки 500мг 30шт,ацетилсалициловый кислота таблетка 500мг 30шт,NaN


## Work with 4 chosen categories

1. Мороженое сливочное, кг
2. Зефир, пастила, кг
3. Футболка детская, шт
4. Джемпер женский, шт

### 1 category

In [ ]:
targetProductCode = 'Мороженое сливочное, кг'

In [ ]:
Y = df_sample['rosstat_name'].apply(lambda x: 1 if x == targetProductCode else 0)
Y.sum() # number of products in a category (>30)

188

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDClassifier
import numpy as np
import operator
import eli5

In [ ]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95, ngram_range=(1, 2))
tfidf_vectorizer.fit_transform(df_sample['product_name_lemm'].values)
X = tfidf_vectorizer.transform(df_sample['product_name_lemm'].values)

In [ ]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear', random_state=0, C=5, penalty='l2',max_iter=1000)

In [ ]:
model=scikit_log_reg.fit(X,Y)

[LibLinear]

In [ ]:
eli5.show_weights(estimator=model,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

Weight?,Feature
+22.671,мороженое
+11.149,сливочный
+7.487,мороженое сливочный
+6.116,пломбир
+3.697,эскимо
+3.556,шоколадный
+3.250,шоколад
+3.095,глазурь
+2.597,70г
+2.490,вафельный


In [ ]:
EN=SGDClassifier(loss='log', penalty='elasticnet')
model2=EN.fit(X,Y)

In [ ]:
eli5.show_weights(estimator=model2,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

In [ ]:
probs = model.predict_proba(X)
probs2 = model2.predict_proba(X)

In [ ]:
# A function which returns goods with highest predicted probability of being in a category
def highest_prob(df, probs, k):
    best_n = np.argsort(probs, axis=0)
    d = pd.DataFrame()
    d['product_name'] = np.flip(df['product_name'].values[best_n[-k:,1]])
    d['prob'] = np.flip(probs[best_n[-k:,1],1])
    return d

In [ ]:
highest_prob(df_sample, probs, 40)

In [ ]:
highest_prob(df_sample, probs2, 20)

In [ ]:
# Now let's try to predict out of sample

In [ ]:
trainid = df_sample['web_price_id'].tolist()
test_df = df_final[~df_final.web_price_id.isin(trainid)]

In [ ]:
testX = tfidf_vectorizer.transform(test_df['product_name_lemm'].values)
testprobs = model.predict_proba(testX)

In [ ]:
testprobs = model.predict(testX)

In [ ]:
testprobs

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# add a new column "proba" to testprobs
test_df['proba']=testprobs
test_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,web_price_id,product_name,product_name_lemm,proba
0,6611458,ИГРОЛЕНД Игрушки интерактивные,игроленд игрушка интерактивный,0
1,5318090,Calvin Klein Performance Футболка,calvin klein performance футболка,0
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,шиповка nike цвет черный артикул rtlaao360101 ...,0
3,6649792,Avocadoffka Кукла,avocadoffka кукла,0
4,7420308,Uniqcute Постельное белье 2 спальный поплин,uniqcute постельный белье спальный поплин,0
...,...,...,...,...
9300025,7184397,Мойка кухонная Omoikiri Taki 54 U IF IN 4973046,мойка кухонный omoikiri taki,0
9300026,8682610,Перчатки Regatta цвет черный Артикул RE036DMJI...,перчатка regatta цвет черный артикул re036dmji...,0
9300028,10806102,R O C S Зубная щетка мягкая 1 шт,зубной щетка мягкий,0
9300029,9063186,Ацетилсалициловая кислота таблетки 500мг 30шт,ацетилсалициловый кислота таблетка 500мг 30шт,0


In [ ]:
test_df['proba'].sum()

1603

In [ ]:
test_df1 = test_df[test_df['proba']==1]
test_df1 = test_df1.assign(Rosstat_name='Мороженое сливочное, кг')
test_df1

,web_price_id,product_name,product_name_lemm,proba,Rosstat_name
9732,141155,Коктейль молочный Большая Кружка Клубника моро...,коктейль молочный большой кружок клубника моро...,1,"Мороженое сливочное, кг"
31221,27197,Елочное украшение Kurt S Adler Рожок с морожен...,елочный украшение kurt adler рожок мороженое а...,1,"Мороженое сливочное, кг"
36352,7110407,Слайм ПЛЮХ Зефирка мороженое контейнер 120г ПЛЮХ,слайм плюх зефирка мороженое контейнер 120г плюха,1,"Мороженое сливочное, кг"
40218,153758,Мороженое Русский Холодъ Пломбир настоящий рож...,мороженое русский холод пломбир настоящий рожо...,1,"Мороженое сливочное, кг"
51549,6753929,Мороженое молочное Bounty кокос в шоколадной г...,мороженое молочный bounty кокос шоколадный гла...,1,"Мороженое сливочное, кг"
...,...,...,...,...,...
9262873,7774943,Маршмеллоу PiratMarmelad Мороженое 200г,маршмеллоу piratmarmelad мороженое 200г,1,"Мороженое сливочное, кг"
9270128,8016013,Мороженое пломбир Филевское с вишней кусочками...,мороженое пломбир филевский вишня кусочек минд...,1,"Мороженое сливочное, кг"
9280308,10337999,Игровой набор Хома Дома Тележка хомодильник дл...,игровой набор хома дом тележка хомодильник мор...,1,"Мороженое сливочное, кг"
9289007,6973977,Мороженое Б Ю Александров Пломбир с кленовым с...,мороженое александров пломбир кленовый сироп м...,1,"Мороженое сливочное, кг"


In [ ]:
test_df1.to_csv('file1.csv')

In [ ]:
testprobs2 = model2.predict_proba(testX)

In [ ]:
highest_prob(test_df, testprobs, 40)

In [ ]:
highest_prob(test_df, testprobs2, 40)

# 2 category

In [ ]:
targetProductCode = 'Зефир, пастила, кг'

In [ ]:
Y = df_sample['rosstat_name'].apply(lambda x: 1 if x == targetProductCode else 0)
Y.sum() # number of products in a category (>30)

224

In [ ]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95, ngram_range=(1, 2))
tfidf_vectorizer.fit_transform(df_sample['product_name_lemm'].values)
X = tfidf_vectorizer.transform(df_sample['product_name_lemm'].values)

In [ ]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear', random_state=0, C=5, penalty='l2',max_iter=1000)

In [ ]:
model=scikit_log_reg.fit(X,Y)

[LibLinear]

In [ ]:
eli5.show_weights(estimator=model,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

Weight?,Feature
+22.673,пастила
+21.813,зефир
+4.261,белевский
+3.752,воздушный
+3.497,яблочный
+3.437,ассорти
+3.402,вкус
+3.168,яблоко
+2.969,ваниль
+2.788,традиция


In [ ]:
EN=SGDClassifier(loss='log', penalty='elasticnet')
model2=EN.fit(X,Y)

In [ ]:
eli5.show_weights(estimator=model2,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

In [ ]:
probs = model.predict_proba(X)
probs2 = model2.predict_proba(X)

In [ ]:
highest_prob(df_sample, probs, 50)

In [ ]:
highest_prob(df_sample, probs2, 20)

In [ ]:
trainid = df_sample['web_price_id'].tolist()
test_df = df_final[~df_final.web_price_id.isin(trainid)]

In [ ]:
testX = tfidf_vectorizer.transform(test_df['product_name_lemm'].values)
testprobs = model.predict_proba(testX)

In [ ]:
testprobs_2 = model.predict(testX)

In [ ]:
testprobs_2

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
test_df['proba']=testprobs_2
test_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,web_price_id,product_name,product_name_lemm,proba
0,6611458,ИГРОЛЕНД Игрушки интерактивные,игроленд игрушка интерактивный,0
1,5318090,Calvin Klein Performance Футболка,calvin klein performance футболка,0
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,шиповка nike цвет черный артикул rtlaao360101 ...,0
3,6649792,Avocadoffka Кукла,avocadoffka кукла,0
4,7420308,Uniqcute Постельное белье 2 спальный поплин,uniqcute постельный белье спальный поплин,0
...,...,...,...,...
9300025,7184397,Мойка кухонная Omoikiri Taki 54 U IF IN 4973046,мойка кухонный omoikiri taki,0
9300026,8682610,Перчатки Regatta цвет черный Артикул RE036DMJI...,перчатка regatta цвет черный артикул re036dmji...,0
9300028,10806102,R O C S Зубная щетка мягкая 1 шт,зубной щетка мягкий,0
9300029,9063186,Ацетилсалициловая кислота таблетки 500мг 30шт,ацетилсалициловый кислота таблетка 500мг 30шт,0


In [ ]:
test_df['proba'].sum()

1254

In [ ]:
test_df2 = test_df[test_df['proba']==1]
test_df2 = test_df2.assign(Rosstat_name='Зефир, пастила, кг')
test_df2

,web_price_id,product_name,product_name_lemm,proba,Rosstat_name
8562,2785197,Петербургский КондитерЪ Зефир в шоколаде натур...,петербургский кондитер зефир шоколад натуральн...,1,"Зефир, пастила, кг"
61872,8831362,Пастила ВкусВилл Яблочная с брусникой без саха...,пастила вкусвилл яблочный брусника сахар 200г,1,"Зефир, пастила, кг"
94890,11106866,Jess Food Пастила Ассорти фруктов и ягод,jess food пастила ассорти фрукт ягода,1,"Зефир, пастила, кг"
99160,8849293,Пастила PastiLab медовая сливовая 250г,пастила pastilab медовый сливовый 250г,1,"Зефир, пастила, кг"
100448,8831422,Зефир ВкусВилл ванильный 80г,зефир вкусвилл ванильный 80г,1,"Зефир, пастила, кг"
...,...,...,...,...,...
9240679,8105982,Белёвская пастильная мануфактура Пастила,белевская пастильный мануфактура пастила,1,"Зефир, пастила, кг"
9253504,5724365,Пастила БЕЛЕВСКАЯ ПАСТИЛА Заварная Ваниль 250 г,пастила белевская пастила заварной ваниль,1,"Зефир, пастила, кг"
9273181,9682816,Бальзам для губ Martinelia зефир,бальзам губа martinelia зефир,1,"Зефир, пастила, кг"
9282117,2784684,Иван поле Пастила яблоко 9 шт,иван поле пастила яблоко,1,"Зефир, пастила, кг"


In [ ]:
test_df2.to_csv('file2.csv')

In [ ]:
testprobs2 = model2.predict_proba(testX)

In [ ]:
highest_prob(test_df, testprobs, 40)

In [ ]:
highest_prob(test_df, testprobs2, 40)

### 3 category

In [ ]:
targetProductCode = 'Футболка детская, шт'

In [ ]:
Y = df_sample['rosstat_name'].apply(lambda x: 1 if x == targetProductCode else 0)
Y.sum() # number of products in a category (>30)

115

In [ ]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95, ngram_range=(1, 2))
tfidf_vectorizer.fit_transform(df_sample['product_name_lemm'].values)
X = tfidf_vectorizer.transform(df_sample['product_name_lemm'].values)

In [ ]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear', random_state=0, C=5, penalty='l2',max_iter=1000)

In [ ]:
model=scikit_log_reg.fit(X,Y)


[LibLinear]

In [ ]:
eli5.show_weights(estimator=model,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

Weight?,Feature
+18.173,футболка
+8.999,мальчик
+6.574,артикул
+5.897,цвет
+5.048,девочка
+4.871,футболка детский
+3.267,футболка мальчик
+3.174,lets
+3.056,детский
+2.971,белый артикул


In [ ]:
EN=SGDClassifier(loss='log', penalty='elasticnet')
model2=EN.fit(X,Y)

In [ ]:
eli5.show_weights(estimator=model2,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

In [ ]:
probs = model.predict_proba(X)
probs2 = model2.predict_proba(X)

In [ ]:
highest_prob(df_sample, probs, 50)

In [ ]:
highest_prob(df_sample, probs2, 20)

In [ ]:
trainid = df_sample['web_price_id'].tolist()
test_df = df_final[~df_final.web_price_id.isin(trainid)]

In [ ]:
testX = tfidf_vectorizer.transform(test_df['product_name_lemm'].values)
testprobs = model.predict_proba(testX)

In [ ]:
testprobs_3 = model.predict(testX)

In [ ]:
test_df['proba']=testprobs_3
test_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,web_price_id,product_name,product_name_lemm,proba
0,6611458,ИГРОЛЕНД Игрушки интерактивные,игроленд игрушка интерактивный,0
1,5318090,Calvin Klein Performance Футболка,calvin klein performance футболка,0
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,шиповка nike цвет черный артикул rtlaao360101 ...,0
3,6649792,Avocadoffka Кукла,avocadoffka кукла,0
4,7420308,Uniqcute Постельное белье 2 спальный поплин,uniqcute постельный белье спальный поплин,0
...,...,...,...,...
9300025,7184397,Мойка кухонная Omoikiri Taki 54 U IF IN 4973046,мойка кухонный omoikiri taki,0
9300026,8682610,Перчатки Regatta цвет черный Артикул RE036DMJI...,перчатка regatta цвет черный артикул re036dmji...,0
9300028,10806102,R O C S Зубная щетка мягкая 1 шт,зубной щетка мягкий,0
9300029,9063186,Ацетилсалициловая кислота таблетки 500мг 30шт,ацетилсалициловый кислота таблетка 500мг 30шт,0


In [ ]:
test_df['proba'].sum()

32128

In [ ]:
test_df3 = test_df[test_df['proba']==1]
test_df3 = test_df3.assign(Rosstat_name='Футболка детская, шт')
test_df3

,web_price_id,product_name,product_name_lemm,proba,Rosstat_name
368,8482639,Футболка Ice Play цвет синий Артикул RTLAAC395...,футболка ice play цвет синий артикул rtlaac395...,1,"Футболка детская, шт"
585,5562496,Богатырь Футболка,богатырь футболка,1,"Футболка детская, шт"
657,1681256,IMRON Футболка,imron футболка,1,"Футболка детская, шт"
689,1681257,IMRON Футболка,imron футболка,1,"Футболка детская, шт"
695,1681258,IMRON Футболка,imron футболка,1,"Футболка детская, шт"
...,...,...,...,...,...
9299880,9684069,Футболка Kaftan,футболка kaftan,1,"Футболка детская, шт"
9299885,9684070,Футболка Kaftan,футболка kaftan,1,"Футболка детская, шт"
9299925,9684071,Футболка Kaftan,футболка kaftan,1,"Футболка детская, шт"
9299926,9684072,Футболка Kaftan,футболка kaftan,1,"Футболка детская, шт"


In [ ]:
test_df3.to_csv('file3.csv')

In [ ]:
testprobs2 = model2.predict_proba(testX)

In [ ]:
highest_prob(test_df, testprobs, 40)

In [ ]:
highest_prob(test_df, testprobs2, 40)

### 4 category

In [ ]:
targetProductCode = 'Джемпер женский, шт'

In [ ]:
Y = df_sample['rosstat_name'].apply(lambda x: 1 if x == targetProductCode else 0)
Y.sum() # number of products in a category v

144

In [ ]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95, ngram_range=(1, 2))
tfidf_vectorizer.fit_transform(df_sample['product_name_lemm'].values)
X = tfidf_vectorizer.transform(df_sample['product_name_lemm'].values)

In [ ]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear', random_state=0, C=5, penalty='l2',max_iter=1000)

In [ ]:
model=scikit_log_reg.fit(X,Y)


[LibLinear]

In [ ]:
eli5.show_weights(estimator=model,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

Weight?,Feature
+11.804,женский
+10.510,свитер
+9.981,толстовка женский
+8.397,джемпер женский
+8.228,одежда
+7.980,толстовка
+7.546,пуловер
+6.952,свитшот женский
+6.490,джемпер
+6.359,свитшот


In [ ]:
EN=SGDClassifier(loss='log', penalty='elasticnet')
model2=EN.fit(X,Y)

In [ ]:
eli5.show_weights(estimator=model2,
                  feature_names= list(tfidf_vectorizer.get_feature_names_out()),
                 top=(50, 5))

In [ ]:
probs = model.predict_proba(X)
probs2 = model2.predict_proba(X)

In [ ]:
highest_prob(df_sample, probs, 50)

In [ ]:
highest_prob(df_sample, probs2, 20)

In [ ]:
trainid = df_sample['web_price_id'].tolist()
test_df = df_final[~df_final.web_price_id.isin(trainid)]

In [ ]:
testX = tfidf_vectorizer.transform(test_df['product_name_lemm'].values)
testprobs = model.predict_proba(testX)

In [ ]:
testprobs_4 = model.predict(testX)

In [ ]:
testprobs_4

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
test_df['proba']=testprobs_4
test_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,web_price_id,product_name,product_name_lemm,proba
0,6611458,ИГРОЛЕНД Игрушки интерактивные,игроленд игрушка интерактивный,0
1,5318090,Calvin Klein Performance Футболка,calvin klein performance футболка,0
2,8482341,Шиповки Nike цвет черный Артикул RTLAAO360101 ...,шиповка nike цвет черный артикул rtlaao360101 ...,0
3,6649792,Avocadoffka Кукла,avocadoffka кукла,0
4,7420308,Uniqcute Постельное белье 2 спальный поплин,uniqcute постельный белье спальный поплин,0
...,...,...,...,...
9300025,7184397,Мойка кухонная Omoikiri Taki 54 U IF IN 4973046,мойка кухонный omoikiri taki,0
9300026,8682610,Перчатки Regatta цвет черный Артикул RE036DMJI...,перчатка regatta цвет черный артикул re036dmji...,0
9300028,10806102,R O C S Зубная щетка мягкая 1 шт,зубной щетка мягкий,0
9300029,9063186,Ацетилсалициловая кислота таблетки 500мг 30шт,ацетилсалициловый кислота таблетка 500мг 30шт,0


In [ ]:
test_df['proba'].sum()

13467

In [ ]:
test_df4 = test_df[test_df['proba']==1]
test_df4 = test_df4.assign(Rosstat_name='Джемпер женский, шт')
test_df4

,web_price_id,product_name,product_name_lemm,proba,Rosstat_name
190,8409215,Mi Kar Свитер,kar свитер,1,"Джемпер женский, шт"
230,8409246,Mi Kar Свитер,kar свитер,1,"Джемпер женский, шт"
231,8409247,Mi Kar Свитер,kar свитер,1,"Джемпер женский, шт"
789,5114136,Джемпер Old Seams цвет черный Артикул OL021EML...,джемпер old seams цвет черный артикул ol021eml...,1,"Джемпер женский, шт"
1016,10557913,Kit Kate Декор для одежды Термотрансфер для од...,kit kate декор одежда термотрансфер одежда тер...,1,"Джемпер женский, шт"
...,...,...,...,...,...
9299454,8980597,Худи PUMA цвет бежевый Артикул RTLAAU909101 Од...,худить puma цвет бежевый артикул rtlaau909101 ...,1,"Джемпер женский, шт"
9299492,2022704,VAY Пуловер,vay пуловер,1,"Джемпер женский, шт"
9299498,7322506,Джемпер Eterna цвет голубой Артикул MP002XM1ZQ...,джемпер eterna цвет голубой артикул mp002xm1zq...,1,"Джемпер женский, шт"
9299538,2024252,Дважды Два Свитер,дважды свитер,1,"Джемпер женский, шт"


In [ ]:
test_df4.to_csv('file4.csv')

In [ ]:
testprobs2 = model2.predict_proba(testX)

In [ ]:
highest_prob(test_df, testprobs, 40)

In [ ]:
highest_prob(test_df, testprobs2, 40)

In [ ]:
import glob

In [ ]:
files = glob.glob('*.csv')
files

['file2.csv', 'file4.csv', 'file3.csv', 'file1.csv']

In [ ]:
files_combined = pd.DataFrame()

for file in files:
    data = pd.read_csv(file)
    data['filename'] = file
    files_combined = pd.concat([files_combined, data])
del files_combined['product_name_lemm']
del files_combined['filename']
del files_combined['proba']
del files_combined['Unnamed: 0']

files_combined

,web_price_id,product_name,Rosstat_name
0,2785197,Петербургский КондитерЪ Зефир в шоколаде натур...,"Зефир, пастила, кг"
1,8831362,Пастила ВкусВилл Яблочная с брусникой без саха...,"Зефир, пастила, кг"
2,11106866,Jess Food Пастила Ассорти фруктов и ягод,"Зефир, пастила, кг"
3,8849293,Пастила PastiLab медовая сливовая 250г,"Зефир, пастила, кг"
4,8831422,Зефир ВкусВилл ванильный 80г,"Зефир, пастила, кг"
...,...,...,...
1598,7774943,Маршмеллоу PiratMarmelad Мороженое 200г,"Мороженое сливочное, кг"
1599,8016013,Мороженое пломбир Филевское с вишней кусочками...,"Мороженое сливочное, кг"
1600,10337999,Игровой набор Хома Дома Тележка хомодильник дл...,"Мороженое сливочное, кг"
1601,6973977,Мороженое Б Ю Александров Пломбир с кленовым с...,"Мороженое сливочное, кг"


In [ ]:
files_combined.to_csv('files_combined.csv',
                      index=False)